<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/matcha_tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🍵 matcha-tts
- [2024.1 **Matcha-TTS: A fast TTS architecture with conditional flow matching**](https://arxiv.org/abs/2309.03199) | [paper code](https://github.com/shivammehta25/Matcha-TTS)

# 摘要：
一种用于快速 TTS 声学建模的新编码器-解码器架构，使用最佳传输条件流匹配 (OT-CFM) 进行训练。与使用分数匹配训练的模型相比，这产生了基于 ODE 的解码器，能够以更少的合成步骤实现高输出质量。仔细的设计选择还确保每个合成步骤都能快速运行。该方法是概率性的、非自回归的，并且无需外部对齐即可从头开始学习说话。与强大的预训练基线模型相比，Matcha-TTS 系统具有最小的内存占用，可以与长语音上最快的模型相媲美，并在听力测试中获得最高的平均意见得分。

一种非自回归神经 TTS 的新方法，它使用条件流匹配(CFM from [2022.10 **Flow Matching for Generative Modeling**](https://arxiv.org/abs/2210.02747))（类似于校正流(Rectified Flow [2022.9 Flow Straight and Fast: Learning to Generate and Transfer Data with Rectified Flow](https://arxiv.org/abs/2209.03003)）来加速基于 ODE 的语音合成。

- Is probabilistic  是概率性的
- Has compact memory footprint 具有紧凑的内存占用
- Sounds highly natural  听起来非常自然
- Is very fast to synthesise from 合成速度非常快

简洁的结构，训练推理快，使用更少额内存空间，

一种基于连续归一化流的概率性、非自回归、快速采样的 TTS 声学模型。主要有两个创新点：
1. 提出了一种改进的编码器-解码器 TTS 结构，该结构在解码器中结合使用 1D CNN 和 Transformer。这减少了内存消耗并且可以快速评估，从而提高综合速度。
2. 使用最优传输条件流匹配 optimal-transport conditional flow matching(OT-CFM) 来训练这些模型，这是一种学习从数据分布中采样的 ODE 的新方法。与传统的 连续时间归一化流 CNF（continuous-time normalising flows ） 和分数匹配概率流 ODE （probability flow ODE） 相比，OT-CFM 定义了从源到目标的更简单的路径，从而能够以比 DPM（Diffusion probabilistic model） 更少的步骤进行准确合成。

实验结果表明，这两种创新都加速了合成，减少了速度和合成质量之间的权衡。尽管速度快且轻量级，Matcha-TTS能够在不需要外部对齐器的情况下学习说话和对齐。与强大的预训练基线模型相比，Matcha-TTS实现了快速合成，并获得了更好的自然度评分。


# 结构：

Matcha-TTS 是一种非自回归的神经文本到语音（TTS）编码器-解码器架构。其架构的概述在图1中提供。文本编码器和时长预测器架构和Grad-TTS一致，但Text encoder使用旋转位置嵌入而不是相对位置嵌入。对齐和时长模型训练遵循MAS和先验损失 $\angle_{enc}$，如Grad-TTS中所述。预测的时长，经过向上取整后，用于上采样（复制）编码器输出的向量，以获得 $\mu$ ，即在给定文本和所选时长的情况下预测的平均声学特征（例如，梅尔频谱图）。这个平均值用于条件化解码器，解码器预测用于合成的向量场 $v_{t}\left(\phi_{t}^{OT}\left(x_{0}\right)\mid\mu;\theta\right)$，但它并不用作初始噪声样本 $x_0$ 的均值（与Grad-TTS不同）。

![image](https://github.com/user-attachments/assets/50e1e3ad-7181-41a5-8cc2-6d6c62473065)


## Text encoder 文本编码器
位置编码使用 RoPE embedding 而不是相对位置 relative embedding, 和Grad-TTS中Text encoder 相同的超参数
- 来自 [2020. **Glow-TTS: A Generative Flow for Text-to-Speech via Monotonic Alignment Search**](https://arxiv.org/abs/2005.11129)
- 来自 [2021. **Grad-TTS: A diffusion probabilistic model for text-to-speech**](https://arxiv.org/abs/2105.06337)

## Duration predictor 持续时间预测器
和Grad-TTS中Duration predictor 相同的超参数
- 来自 [2020. **Glow-TTS: A Generative Flow for Text-to-Speech via Monotonic Alignment Search**](https://arxiv.org/abs/2005.11129)
- 来自 [2021. **Grad-TTS: A diffusion probabilistic model for text-to-speech**](https://arxiv.org/abs/2105.06337)


## Decoder (the Flow-prediction network 流预测网络)
使用两个下采样块，然后是两个中采样块和两个上采样块，如图 2 所示。每个块都有一个 Transformer 层，隐藏维度为 256、2 个头、注意力维度为 64，并且使用snake 激活函数。

![image](https://github.com/user-attachments/assets/5c50f129-e7bf-427e-aa75-cfd01f6b089b)

- 受LDM [2021. High-resolution image synthesis with latent diffusion models](https://arxiv.org/abs/2112.10752)的启发,一个包含 1D 卷积残差块的 U-Net，用于对输入进行下采样和上采样，并具有流匹配步骤 t∈[0,1] 嵌入, 每个残差块后面都有一个 Transformer块，其前馈网络使用 snake 激活函数，这些 Transformer 不使用任何位置嵌入，因为音素(phone)之间的位置信息已经由编码器嵌入，并且卷积和下采样操作用于在同一音素内的帧之间，插入这些信息并区分它们彼此的相对位置。与 Grad-TTS 使用的仅 2D 卷积 U-Net 相比，该解码器网络的评估速度明显更快，并且消耗的内存更少。

- 激活函数： snake active function 来自 [2020. Neural networks fail to learn periodic functions and how to fix it](https://arxiv.org/abs/2006.08195) 和 [2022. BigVGAN: A universal neural vocoder with large-scale training](https://arxiv.org/abs/2206.04658) 一样的使用方法
- 结构来自 [2021. **Grad-TTS: A diffusion probabilistic model for text-to-speech**](https://arxiv.org/abs/2105.06337)


## Optimal-transport conditional flow matching 最优传输条件流匹配
在这一部分，首先介绍由向量场生成的概率密度路径，然后引入提出的方法中使用的最优传输条件流匹配（OT-CFM）目标。
设 $\mathbf{x}$ 表示数据空间 $\mathbb{R}^d$中的一个观测值，它从复杂且未知的数据分布 $q(\mathbf{x}) q(x)$ 中采样得到。一个概率密度路径是一个随时间变化的概率密度函数， $p_t: [0,1] \times \mathbb{R}^d \rightarrow \mathbb{R}^{>0} pt:[0,1]×Rd→R>0 $，其中 $t \in [0,1] t∈[0,1]$ ，
$p_0(\mathbf{x}) = \mathcal{N}(\mathbf{x}; \mathbf{0}, \mathbf{I})$ 是一个先验分布，使得 $p_1(\mathbf{x})$ 近似于数据分布 $q(\mathbf{x})$ 。
例如，连续时间归一化流（CNFs）首先定义一个向量场 $\mathbf{v}_t: [0,1] \times \mathbb{R}^d \rightarrow \mathbb{R}^d$ ，它通过下面的常微分方程（ODE）生成流 $\phi_t: [0,1] \times \mathbb{R}^d \rightarrow \mathbb{R}^d$ ：
$$\frac{d}{dt} \phi_t(\mathbf{x}) = \mathbf{v}_t(\phi_t(\mathbf{x})) \quad ; \quad \phi_0(\mathbf{x}) = \mathbf{x}.$$
这生成了路径 $p_t$ ，作为数据点的边际概率分布。我们可以通过解决方程（1）中的初值问题，从近似的数据分布 $p_1$ 中采样。 假设存在一个已知的向量场 $\mathbf{u}_t$ ，它从 $p_0$ 到 $p_1 \approx q$ 生成概率路径 $p_t$ 。流匹配损失定义为：
$$\mathcal{L}_{\mathrm{FM}}(\theta) = \mathbb{E}_{t,p_t(\mathbf{x})} \left\| \mathbf{u}_t(\mathbf{x}) - \mathbf{v}_t(\mathbf{x}; \theta) \right\|^2,$$
其中 $t \sim \mathbb{U}[0,1]$ 并且 $\mathbf{v}_t(\mathbf{x}; \theta)$ 是一个具有参数 $\theta$ 的神经网络。然而，流匹配在实践中是不可行的，因为获取向量场 $\mathbf{u}_t$ 和目标概率 $p_t$ 并非易事。因此，条件流匹配考虑的是：
$$\mathcal{L}_{\mathrm{CFM}}(\theta) = \mathbb{E}_{t,q(\mathbf{x}_1),p_t(\mathbf{x} \mid \mathbf{x}_1)} \left\| \mathbf{u}_t(\mathbf{x} \mid \mathbf{x}_1) - \mathbf{v}_t(\mathbf{x}; \theta) \right\|^2.$$
这替换了难以处理的边际概率密度和向量场，转而考虑条件概率密度和条件向量场。关键是，这些通常是可行的，并且有封闭形式的解决方案，而且可以证明 $\mathcal{L}_{\mathrm{CFM}}(\theta)$ 和 $\mathcal{L}_{\mathrm{FM}}(\theta)$ 在 $\theta$ 方面具有相同的梯度。
在Matcha-TTS的情况下，$\mathbf{x}_1$ 是声学帧，而 $\boldsymbol{\mu}$ 是从文本预测的这些帧的条件均值，使用下一部分描述的架构。 $\sigma_{\min}$ 是一个具有很小值（在我们的实验中为1e-4）的超参数。
- 来自 [2022.10 Flow Matching for Generative Modeling](https://arxiv.org/abs/2210.02747)

# 训练
常规数据集，论文中使用单人数据（LJ Speech），额外也用多人数据（VCTK）来训练；可以使用lhotse工具下载, 进行切分数据预处理，然后通过带有 espeak-ng 后端的phonemizer: https://github.com/bootphon/phonemizer 用于将输入字素（graphemes）转换为 IPA 音素（IPA phones），在 2 个 GPU 上训练了 500k 更新，批量大小为 32，学习率为 1e-4。

## 训练数据集：
- [LJ Speech](https://keithito.com/LJ-Speech-Dataset/)
- [VCTK](https://datashare.ed.ac.uk/handle/10283/2651)

## 对比开源模型
- [2020. FastSpeech 2: Fast and High-Quality End-to-End Text to Speech](https://arxiv.org/abs/2006.04558) 一种快速非-概率声学模型 | [非官方 code](https://github.com/ming024/FastSpeech2)， Matcha-TTS的论文中提到用，改为使用 Meta 的 https://github.com/facebookresearch/fairseq 中的检查点（ckpt）?；
- [2021.6 **VITS: Conditional Variational Autoencoder with Adversarial Learning for End-to-End Text-to-Speech**](https://arxiv.org/abs/2106.06103) 一个具有离散时间归一化流的强大概率端对端 TTS 系统 | [paper code](https://github.com/jaywalnut310/vits)
- [2021.5 Grad-TTS: A Diffusion Probabilistic Model for Text-to-Speech](https://arxiv.org/abs/2105.06337) 一种基于 DPM 的强大声学模型 | [paper code](https://github.com/huawei-noah/Speech-Backbones/tree/main/Grad-TTS)

## 对比评估
从内存使用量(模型参数)，推理速度，以及生成语音质量三个方面评估：
- 所有系统训练期间（批量大小为 32 和 fp16）的参数数量和最大内存使用情况，使用内存小;
- 在训练系统之后，我们通过计算在测试集上合成时的实时因子（RTF）的平均值和标准差，以及将Whisper medium ASR系统应用于结果时的单词错误率（WER），来评估不同条件的合成速度和可理解性；
- 平均意见得分（MOS）听力测试(众包人工测试)，这个比较意义不大；
- 评估了不同模型的合成速度如何随着语句长度的增加而变化，通过使用GPT-2模型生成不同长度的180个句子，评估合成时间，对长句子合成更快；

------

结合作者以前的研究成果，都是在3090上进行训练，构造精简小而美的结构，满足基本要求(正常文本到语音)
- [2021.8 Neural HMMs are all you need (for high-quality attention-free TTS)](https://arxiv.org/abs/2108.13320) | [paper code](https://github.com/shivammehta25/Neural-HMM)
- [2022.11 OverFlow: Putting flows on top of neural transducers for better TTS](https://arxiv.org/abs/2211.06892) | [paper code](https://github.com/shivammehta25/OverFlow)


# 总结：

论文实现的代码结构清晰，提炼的网络结构参数少，所需硬件成本不高，挺适合快速上手训练一个tts，然后根据需求和所要解决的实际问题，加入其他网络结构，再去推进升级。比如 VALL-E, Seed-TTS(非纯DiT结构)，CosyVoice 基于大语言模型（LLM）的文本转语音（TTS）成为主流，生成更加泛化(涌现),但是所需训练数据和模型网络参数大(scaling)，训练推理成本高(虽然有大量的推理优化工作)。 [2024.7 CosyVoice: A Scalable Multilingual Zero-shot Text-to-speech Synthesizer based on Supervised Semantic Tokens](https://arxiv.org/abs/2407.05407) | [2024.12 CosyVoice 2: Scalable Streaming Speech Synthesis with Large Language Models](https://arxiv.org/abs/2412.10117) | [paper code](https://github.com/FunAudioLLM/CosyVoice)

In [ ]:
!apt-get install espeak-ng 2>&1 1>/dev/null

In [ ]:
!git clone https://github.com/shivammehta25/Matcha-TTS.git

In [ ]:
!cd Matcha-TTS && pip install -q -e .

In [ ]:
!pip show matplotlib

In [ ]:
!pip install matplotlib==3.7.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 104.1 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    Uninstalling matplotlib-3.10.0:
      Successfully uninstalled matplotlib-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.31.0 requires matplotlib>=3.7.1, but you have matplotlib 3.7.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.0 which is incompatible.


## dataset

In [ ]:
!pip install -q git+https://github.com/lhotse-speech/lhotse

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.8 MB/s eta 0:00:00


In [ ]:
!lhotse download ljspeech /content/data

/content/data/LJSpeech-1.1.tar.bz2: 100% 2.75G/2.75G [00:17<00:00, 156MB/s]


In [ ]:
!lhotse prepare ljspeech  /content/data/LJSpeech-1.1  /content/data/manifests

2025-01-14 13:08:23,636 WARNING [qa.py:197] Trimmed 6561 supervisions exceeding the end of the recording.


In [ ]:
!cat /content/data/LJSpeech-1.1/metadata.csv | wc -l
!head  /content/data/LJSpeech-1.1/metadata.csv
!tail  /content/data/LJSpeech-1.1/metadata.csv

13100
LJ001-0001|Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition|Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition
LJ001-0002|in being comparatively modern.|in being comparatively modern.
LJ001-0003|For although the Chinese took impressions from wood blocks engraved in relief for centuries before the woodcutters of the Netherlands, by a similar process|For although the Chinese took impressions from wood blocks engraved in relief for centuries before the woodcutters of the Netherlands, by a similar process
LJ001-0004|produced the block books, which were the immediate predecessors of the true printed book,|produced the block books, which were the immediate predecessors of the true printed book,
LJ001-0005|the invention of movable metal letters in the middle of the fifteenth century may

In [ ]:
from IPython.display import Audio
Audio('/content/data/LJSpeech-1.1/wavs/LJ001-0001.wav')

In [ ]:
!wget https://raw.githubusercontent.com/NVIDIA/tacotron2/refs/heads/master/filelists/ljs_audio_text_train_filelist.txt -O /content/data/ljs_audio_text_train_filelist.txt
!wget https://raw.githubusercontent.com/NVIDIA/tacotron2/refs/heads/master/filelists/ljs_audio_text_val_filelist.txt -O /content/data/ljs_audio_text_val_filelist.txt
!wget https://raw.githubusercontent.com/NVIDIA/tacotron2/refs/heads/master/filelists/ljs_audio_text_test_filelist.txt -O /content/data/ljs_audio_text_test_filelist.txt


--2025-01-14 13:20:41--  https://raw.githubusercontent.com/NVIDIA/tacotron2/refs/heads/master/filelists/ljs_audio_text_train_filelist.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1524164 (1.5M) [text/plain]
Saving to: ‘/content/data/ljs_audio_text_train_filelist.txt’

/content/data/ljs_a 100%[===================>]   1.45M  --.-KB/s    in 0.07s   

2025-01-14 13:20:41 (22.2 MB/s) - ‘/content/data/ljs_audio_text_train_filelist.txt’ saved [1524164/1524164]

--2025-01-14 13:20:41--  https://raw.githubusercontent.com/NVIDIA/tacotron2/refs/heads/master/filelists/ljs_audio_text_val_filelist.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw

In [ ]:
!head /content/data/ljs_audio_text_train_filelist.txt


DUMMY/LJ050-0234.wav|It has used other Treasury law enforcement agents on special experiments in building and route surveys in places to which the President frequently travels.
DUMMY/LJ019-0373.wav|to avail himself of his powers, as it was difficult to bring home the derelictions of duties and evasion of the acts. Too much was left to the inspectors.
DUMMY/LJ050-0207.wav|Although Chief Rowley does not complain about the pay scale for Secret Service agents,
DUMMY/LJ048-0203.wav|The three officers confirm that their primary concern was crowd and traffic control,
DUMMY/LJ003-0182.wav|The tried and the untried, young and old, were herded together
DUMMY/LJ044-0166.wav|According to Marina Oswald, he thought that would help him when he got to Cuba.
DUMMY/LJ019-0208.wav|The proposal made was to purchase some fifty thousand square feet between Newgate, Warwick Lane, and the Sessions House,
DUMMY/LJ021-0146.wav|I shall seek assurances of the making and maintenance of agreements, which can be mut

In [ ]:
!sed -i -- 's,DUMMY,/content/data/LJSpeech-1.1/wavs,g' /content/data/ljs_audio_text_*_filelist.txt

In [ ]:
!head /content/data/ljs_audio_text_train_filelist.txt
print("=="*10)
!head /content/data/ljs_audio_text_val_filelist.txt
print("=="*10)
!head /content/data/ljs_audio_text_test_filelist.txt

/content/data/LJSpeech-1.1/wavs/LJ050-0234.wav|It has used other Treasury law enforcement agents on special experiments in building and route surveys in places to which the President frequently travels.
/content/data/LJSpeech-1.1/wavs/LJ019-0373.wav|to avail himself of his powers, as it was difficult to bring home the derelictions of duties and evasion of the acts. Too much was left to the inspectors.
/content/data/LJSpeech-1.1/wavs/LJ050-0207.wav|Although Chief Rowley does not complain about the pay scale for Secret Service agents,
/content/data/LJSpeech-1.1/wavs/LJ048-0203.wav|The three officers confirm that their primary concern was crowd and traffic control,
/content/data/LJSpeech-1.1/wavs/LJ003-0182.wav|The tried and the untried, young and old, were herded together
/content/data/LJSpeech-1.1/wavs/LJ044-0166.wav|According to Marina Oswald, he thought that would help him when he got to Cuba.
/content/data/LJSpeech-1.1/wavs/LJ019-0208.wav|The proposal made was to purchase some fifty 

In [ ]:
from IPython.display import Audio
Audio('/content/data/LJSpeech-1.1/wavs/LJ050-0234.wav')

In [ ]:
from IPython.display import Audio
Audio('/content/data/LJSpeech-1.1/wavs/LJ022-0023.wav')

In [ ]:
from IPython.display import Audio
Audio('/content/data/LJSpeech-1.1/wavs/LJ045-0096.wav')

In [ ]:
!cat /content/Matcha-TTS/configs/data/ljspeech.yaml

_target_: matcha.data.text_mel_datamodule.TextMelDataModule
name: ljspeech
train_filelist_path: /content/data/ljs_audio_text_train_filelist.txt
valid_filelist_path: /content/data/ljs_audio_text_val_filelist.txt
batch_size: 32
num_workers: 20
pin_memory: True
cleaners: [english_cleaners2]
add_blank: True
n_spks: 1
n_fft: 1024
n_feats: 80
sample_rate: 22050
hop_length: 256
win_length: 1024
f_min: 0
f_max: 8000
data_statistics:  # Computed for ljspeech dataset
  mel_mean: -5.536622
  mel_std: 2.116101
seed: ${seed}
load_durations: false


In [ ]:
# Generate normalisation statistics with the yaml file of dataset configuration
!matcha-data-stats -i ljspeech.yaml -f


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
{'mel_mean': -5.517028331756592, 'mel_std': 2.064394474029541}


In [ ]:
!cat ljspeech.json

{"mel_mean": -5.517028331756592, "mel_std": 2.064394474029541}

In [ ]:
!cat /content/Matcha-TTS/configs/data/ljspeech.yaml

_target_: matcha.data.text_mel_datamodule.TextMelDataModule
name: ljspeech
train_filelist_path: /content/data/ljs_audio_text_train_filelist.txt
valid_filelist_path: /content/data/ljs_audio_text_val_filelist.txt
batch_size: 32
num_workers: 20
pin_memory: True
cleaners: [english_cleaners2]
add_blank: True
n_spks: 1
n_fft: 1024
n_feats: 80
sample_rate: 22050
hop_length: 256
win_length: 1024
f_min: 0
f_max: 8000
data_statistics:  # Computed for ljspeech dataset
  mel_mean: -5.517028
  mel_std: 2.064394
seed: ${seed}
load_durations: false


## train

In [ ]:
!make -C /content/Matcha-TTS/ train-ljspeech

流式输出内容被截断，只能显示最后 5000 行内容。
Epoch 177/-2 ━━━━━━━╺━━━━━━━━━━━━━━━━━ 117/391 0:01:31 • 0:03:21 1.37it/s v_num: 0.000 step:        
                                                                          69322.000 loss/train_step:
                                                                          2.240 loss/val_step: 2.010
                                                                          loss/val_epoch: 2.236     
Epoch 177/-2 ━━━━━━━╺━━━━━━━━━━━━━━━━━ 117/391 0:01:31 • 0:03:21 1.37it/s v_num: 0.000 step:        
                                                                          69323.000 loss/train_step:
                                                                          2.334 loss/val_step: 2.010
                                                                          loss/val_epoch: 2.236     
Epoch 177/-2 ━━━━━━━╸━━━━━━━━━━━━━━━━━ 118/391 0:01:32 • 0:03:20 1.37it/s v_num: 0.000 step:        
                                                                

In [ ]:
# for a minimum memory run
!python /content/Matcha-TTS/matcha/train.py experiment=ljspeech_min_memory

## upload ckpt

In [ ]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')


In [ ]:
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential

In [ ]:
!huggingface-cli upload --repo-type model weege007/matchaTTS  /content/Matcha-TTS/logs/train/ljspeech/runs/2025-01-14_13-47-46/checkpoints/last.ckpt ljspeech.ckpt

## Synthesise from the custom trained model

In [ ]:
!matcha-tts --text "show me the money" --checkpoint_path /content/Matcha-TTS/logs/train/ljspeech/runs/2025-01-14_13-47-46/checkpoints/last.ckpt --vocoder hifigan_T2_v1

## onnx

In [ ]:
!pip install onnx


In [ ]:
!cd /content/Matcha-TTS/ && python -m matcha.onnx.export \
  /content/Matcha-TTS/logs/train/ljspeech/runs/2025-01-14_13-47-46/checkpoints/last.ckpt model.onnx \
  --n-timesteps 5


In [ ]:
# add hifigan vocoder to model.onnx
!cd /content/Matcha-TTS/ && python -m matcha.onnx.export \
  /content/Matcha-TTS/logs/train/ljspeech/runs/2025-01-14_13-47-46/checkpoints/last.ckpt model.onnx \
  --vocoder-name hifigan_T2_v1 \
  --vocoder-checkpoint-path /root/.local/share/matcha_tts/hifigan_T2_v1 \
  --n-timesteps 5


In [ ]:
#!pip install onnxruntime
!pip install onnxruntime-gpu  # for GPU inference


In [ ]:
!cd /content/Matcha-TTS/ && python -m matcha.onnx.infer model.onnx --text "hey" --output-dir ./outputs --temperature 0.4 --spk 0


In [ ]:
# cpu infer with onnxrutime
!cd /content/Matcha-TTS/ && python -m matcha.onnx.infer model.onnx --text "hey" --output-dir ./outputs


TIP:

If you exported only Matcha to ONNX, this will write mel-spectrogram as graphs and numpy arrays to the output directory. If you embedded the vocoder in the exported graph, this will write .wav audio files to the output directory.

如果您仅将 Matcha 导出到 ONNX，这会将 mel-spectrogram 作为图形和numpy数组写入输出目录。如果您将声码器嵌入到导出的图形中，这会将.wav音频文件写入输出目录。



In [ ]:
# gpu infer with onnxrutime
# if
!cd /content/Matcha-TTS/ && python -m matcha.onnx.infer model.onnx --text "hey" --output-dir ./outputs --gpu


In [ ]:
# if model.onnx no hifigan vocoder, use other, eg: hifigan.small.onnx
!cd /content/Matcha-TTS/ && python -m matcha.onnx.infer model.onnx --text "hey" --output-dir ./outputs --vocoder hifigan.small.onnx --gpu

## Extract phoneme alignments from Matcha-TTS


In [ ]:
!cd /content/Matcha-TTS/ && python matcha/utils/get_durations_from_trained_model.py -i ljspeech.yaml \
  -c /content/Matcha-TTS/logs/train/ljspeech/runs/2025-01-14_13-47-46/checkpoints/last.ckpt

In [ ]:
#